In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import sqlite3

conn = sqlite3.connect('/content/drive/MyDrive/chat/products.db')
cursor = conn.cursor()

In [3]:
cursor.execute('''
CREATE TABLE IF NOT EXISTS categories (
    id INTEGER PRIMARY KEY,
    name TEXT NOT NULL,
    description TEXT
)
''')

cursor.execute('''
CREATE TABLE IF NOT EXISTS products (
    id INTEGER PRIMARY KEY,
    name TEXT NOT NULL,
    description TEXT,
    category_id INTEGER,
    price REAL,
    stock_quantity INTEGER,
    FOREIGN KEY (category_id) REFERENCES categories(id)
)
''')

conn.commit()
conn.close()

In [4]:
import sqlite3

conn = sqlite3.connect('/content/drive/MyDrive/chat/products.db')
cursor = conn.cursor()


In [5]:
import random

In [6]:
# Đọc dữ liệu từ HuggingFace parquet
import pandas as pd
df = pd.read_parquet("hf://datasets/ShynBui/shopee_products_classification/data/train-00000-of-00001.parquet").sample(200)

df = df.dropna(subset=["id", "name", "description", "label_id", "class_name"])

df['label_id'] = df['label_id'].astype(int)
df['id'] = df['id'].astype(str)

categories_df = df[['label_id', 'class_name']].drop_duplicates().rename(
    columns={'label_id': 'id', 'class_name': 'name'}
)
categories_df['description'] = "Danh mục " + categories_df['name'].str.lower()

products_df = df[['id', 'name', 'description', 'label_id']].copy()
products_df = products_df.rename(columns={'label_id': 'category_id'})
products_df['price'] = products_df['id'].apply(lambda x: round(random.uniform(50_000, 1_500_000), 2))
products_df['stock_quantity'] = products_df['id'].apply(lambda x: random.randint(0, 100))

conn = sqlite3.connect('/content/drive/MyDrive/chat/products.db')
cursor = conn.cursor()

categories_df.to_sql('categories', conn, if_exists='replace', index=False)

products_df.to_sql('products', conn, if_exists='replace', index=False)

print("📂 Danh mục:")
print(pd.read_sql("SELECT * FROM categories LIMIT 5", conn))

print("\n📦 Sản phẩm:")
print(pd.read_sql("SELECT * FROM products LIMIT 5", conn))

categories_df.to_sql('categories', conn, if_exists='append', index=False)
products_df.to_sql('products', conn, if_exists='append', index=False)

conn.commit()
conn.close()

print("Dữ liệu đã được lưu vào SQLite database thành công.")

conn.close()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
<ipython-input-6-a9e401292b0e>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['label_id'] = df['label_id'].astype(int)
<ipython-input-6-a9e401292b0e>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a

📂 Danh mục:
   id                     name                       description
0  24                   Xe Đạp                   Danh mục xe đạp
1  23                   Xe Máy                   Danh mục xe máy
2  21       Thể Thao & Du Lịch       Danh mục thể thao & du lịch
3  20                  Đồ Chơi                  Danh mục đồ chơi
4   8  Phụ Kiện & Trang Sức Nữ  Danh mục phụ kiện & trang sức nữ

📦 Sản phẩm:
                                              id  \
0     cavailles_shopee_322_20058066408_shopee.vn   
1        tuborg_shopee_322_16428770410_shopee.vn   
2      cavailles_shopee_322_3424303237_shopee.vn   
3  waterpush_shopee_pub_322_9886900668_shopee.vn   
4                    tikivn_62_191356977_tiki.vn   

                                                name  \
0  Cối líp thả xe đạp ổ bi bạc đạn thay thế cho c...   
1  Tấm cách nhiệt ống xả B HONDA Sh 125 / 150 201...   
2                             Mỡ bôi trơn xe đạp 30G   
3        Mỡ bôi trơn chịu nhiệt xe đạp 50g (Mỡ X

In [7]:
import sqlite3
import pandas as pd


conn = sqlite3.connect('/content/drive/MyDrive/chat/products.db')


categories_df = pd.read_sql_query("SELECT * FROM categories", conn)
print("📂 Danh mục sản phẩm:")
print(categories_df.head(15))


products_df = pd.read_sql_query("SELECT * FROM products", conn)
print("\n📦 Sản phẩm:")
print(products_df.head(10))

conn.close()


📂 Danh mục sản phẩm:
    id                     name                       description
0   24                   Xe Đạp                   Danh mục xe đạp
1   23                   Xe Máy                   Danh mục xe máy
2   21       Thể Thao & Du Lịch       Danh mục thể thao & du lịch
3   20                  Đồ Chơi                  Danh mục đồ chơi
4    8  Phụ Kiện & Trang Sức Nữ  Danh mục phụ kiện & trang sức nữ
5    3        Thời Trang Trẻ Em        Danh mục thời trang trẻ em
6   18                 Sức Khỏe                 Danh mục sức khỏe
7    6                Túi Ví Nữ                Danh mục túi ví nữ
8    4              Giày Dép Nữ              Danh mục giày dép nữ
9    1           Thời Trang Nam           Danh mục thời trang nam
10  17                  Sắc Đẹp                  Danh mục sắc đẹp
11  25          Bách Hóa Online          Danh mục bách hóa online
12   7        Balo & Túi Ví Nam        Danh mục balo & túi ví nam
13  12  Máy Ảnh & Máy Quay Phim  Danh mục máy ảnh & máy

In [8]:
len(products_df)

222